In [9]:
import pandas as pd

In [2]:
!python3 -m pip install pandas

  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 10.9 MB/s eta 0:00:00a 0:00:01
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


README.txt    links.csv     ratings.csv
checksums.txt movies.csv    tags.csv


In [39]:
# Datei-Pfade anpassen!
movies = pd.read_csv("ml-32m/movies.csv")
ratings = pd.read_csv("ml-32m/ratings.csv")
tags = pd.read_csv("ml-32m/tags.csv")

# Daten anschauen
print(movies.head())
print(ratings.head())
print(tags.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228
2       1       29     2.0  943230976
3       1       30     5.0  944249077
4       1       32     5.0  943228858
   userId  movieId          tag   timestamp
0      22    26479  Kevin Kline  1583038886
1      22    79592     misogyny  1581476297
2      22   247150   acrophobia  1622483469
3

In [28]:
print(movies.columns)
print(ratings.columns)
print(tags.columns)

Index(['movieId', 'title', 'genres'], dtype='object')
Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')


In [23]:
for col in movies.columns:
    print(col)

movieId
title
genres


In [29]:
movies.info()
ratings.info()
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  87585 non-null  int64 
 1   title    87585 non-null  object
 2   genres   87585 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000204 entries, 0 to 32000203
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 976.6 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000072 entries, 0 to 2000071
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   userId     int64 
 1   movieId    int64 
 2   tag        object
 3   timestamp  int64 
dtypes: int64(3), object(1)
memory usage: 61.0+ MB


In [49]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [32]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [48]:
tags.isnull().sum()

userId        0
movieId       0
tag          17
timestamp     0
dtype: int64

In [50]:
# Clean tags csv file before merging
tags = tags.dropna(subset=['tag'])

In [51]:
tags.isnull().sum()

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

In [35]:
movies.nunique()
# Anzahl einzigartiger WErte

movieId    87585
title      87382
genres      1798
dtype: int64

In [36]:
movies['genres'].value_counts().head(10)
# häufigsten Genres

genres
Drama                   12443
Documentary              8132
Comedy                   7761
(no genres listed)       7080
Comedy|Drama             3245
Drama|Romance            2825
Horror                   2487
Comedy|Romance           2229
Thriller                 1410
Comedy|Drama|Romance     1335
Name: count, dtype: int64

In [37]:
movies['title'].unique()[:10]
# ersten 10 einzigartigen Titel

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       'Waiting to Exhale (1995)', 'Father of the Bride Part II (1995)',
       'Heat (1995)', 'Sabrina (1995)', 'Tom and Huck (1995)',
       'Sudden Death (1995)', 'GoldenEye (1995)'], dtype=object)

In [54]:
# movies mit ratings verbinden
movies_ratings = pd.merge(movies, ratings, on='movieId', how='inner')

# Das Ergebnis dann noch mit tags verbinden
movies_ratings_tags = pd.merge(movies_ratings, tags, on=['movieId', 'userId'], how='left')

# Ein Blick auf das Ergebnis
movies_ratings_tags.info()
movies_ratings_tags.head()


# nur Filme die bewertet und getagged wurden
movies_complete = pd.merge(movies_ratings, tags, on=['movieId', 'userId'], how='inner')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33178177 entries, 0 to 33178176
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   movieId      int64  
 1   title        object 
 2   genres       object 
 3   userId       int64  
 4   rating       float64
 5   timestamp_x  int64  
 6   tag          object 
 7   timestamp_y  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 2.0+ GB


In [40]:
movies_ratings_tags['tag'].value_counts().head(10)


tag
sci-fi                10508
atmospheric            9321
action                 8077
comedy                 7571
funny                  7260
surreal                6974
visually appealing     6927
twist ending           6349
thought-provoking      5846
dark comedy            5657
Name: count, dtype: int64

In [56]:
movies_complete.head()

,movieId,title,genres,userId,rating,timestamp_x,tag,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,109,4.0,1254982916,children,1257988285
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,109,4.0,1254982916,Disney,1257988287
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,909,2.0,1503218498,animation,1248249498
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,909,2.0,1503218498,children,1248249511
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,909,2.0,1503218498,Disney,1248249494


In [43]:
# Welche Filme haben keine Bewertungen?
no_ratings = movies[~movies['movieId'].isin(ratings['movieId'])]
print(f"Anzahl Filme ohne Bewertungen: {len(no_ratings)}")

# Welche Filme haben keine Tags?
no_tags = movies[~movies['movieId'].isin(tags['movieId'])]
print(f"Anzahl Filme ohne Tags: {len(no_tags)}")


Anzahl Filme ohne Bewertungen: 3153
Anzahl Filme ohne Tags: 36262


In [44]:
# Anzahl der Tags pro Film
tags_per_movie = tags.groupby('movieId')['tag'].count().reset_index(name='tag_count')

# Statistik: Anzahl der Tags pro Film
print(tags_per_movie['tag_count'].describe())

# Filme mit nur einem Tag anzeigen (zur Kontrolle)
single_tag_movies = tags_per_movie[tags_per_movie['tag_count'] == 1]
print(f"Anzahl Filme mit nur einem Tag: {len(single_tag_movies)}")


count    51323.000000
mean        38.969955
std        160.935295
min          1.000000
25%          2.000000
50%          5.000000
75%         16.000000
max       6697.000000
Name: tag_count, dtype: float64
Anzahl Filme mit nur einem Tag: 7682


In [52]:
# Filme, die mindestens eine Bewertung und einen Tag haben
movies_with_all = movies[
    (movies['movieId'].isin(ratings['movieId'])) & 
    (movies['movieId'].isin(tags['movieId']))
]

print(f"number of movies with ratings and tags: {len(movies_with_all)}")


number of movies with ratings and tags: 48170


In [57]:
movies_with_all.describe(include='all')

,movieId,title,genres
count,48170.000000,48170,48170
unique,NaN,48119,1587
top,NaN,Journey to the Center of the Earth (2008),Drama
freq,NaN,3,7124
mean,119479.316483,NaN,NaN
std,73845.758549,NaN,NaN
min,1.000000,NaN,NaN
25%,61829.000000,NaN,NaN
50%,129946.000000,NaN,NaN
75%,172778.500000,NaN,NaN
